# PyCitySchools

### Data Analysis of sample district-wide standarized test results for math and reading scores (9th-12th grade)

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
## INITIALIZE & LOAD DATA

# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [2]:
## CLEAN, ORGANIZE AND COMPACT DATA

## Rename Columns for School Data DataFrame
school_data = school_data.rename(columns={
    "school_name": "School Name",
    "type": "School Type",
    "size": "Total Students",
    "budget": "Total School Budget"
})

## Rename Columns for Complete School Data DataFrame
school_data_complete = school_data_complete.rename(columns={
    "student_name": "Student Name",
    "gender": "Gender",
    "grade": "Grade",
    "school_name": "School Name",
    "reading_score": "Reading Score",
    "math_score": "Math Score",
    "type": "School Type",
    "size": "Total Students",
    "budget": "Total School Budget"
})

## Build School Analysis DataFrame
school_analysis = school_data_complete[[
    "Student Name",
    "Gender",
    "Grade",
    "School Name",
    "Reading Score",
    "Math Score",
    "School Type"
]]

In [3]:
## MATH, READING & OVERALL PASS VALIDATION (BOOLEANS)

## Validate Score Pass (with boolean) if [Score >= 70]
math_pass = school_analysis["Math Score"]>=70
reading_pass = school_analysis["Reading Score"]>=70
overall_pass = (school_analysis["Reading Score"]>=70) & (school_analysis["Math Score"]>=70)

## Insert Pass Validation into School Analysis DataFrame
school_analysis.insert(len(school_analysis.columns), "Math Pass", math_pass, allow_duplicates=False)
school_analysis.insert(len(school_analysis.columns), "Reading Pass", reading_pass, allow_duplicates=False)
school_analysis.insert(len(school_analysis.columns), "Overall Pass", overall_pass, allow_duplicates=False)

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [4]:
## DISTRICT SUMMARY

## 1. Compute District Summary

## Total number of schools
total_schools = len(school_data["School Name"])
## Total number of students
total_students = school_data["Total Students"].sum()
## Total budget
total_budget = school_data["Total School Budget"].sum()
## Average math score
avg_math_score = school_analysis["Math Score"].mean()
## Average reading score
avg_reading_score = school_analysis["Reading Score"].mean()
## Percentage of students with a passing math score (70 or greater)
pct_math_pass = school_analysis["Math Pass"].sum() / total_students
## Percentage of students with a passing reading score (70 or greater)
pct_reading_pass = school_analysis["Reading Pass"].sum() / total_students
## Percentage of students who passed math and reading (% Overall Passing)
pct_overall_pass = school_analysis["Overall Pass"].sum() / total_students

## 2. Create & Display District Summary DataFrame
district_summary = pd.DataFrame({
    "Total Schools": [total_schools],
    "Total Students": [total_students],
    "Total Budget": [total_budget],
    "Average Math Score": [avg_math_score],
    "Average Reading Score": [avg_reading_score],
    "% Passing Math": [pct_math_pass],
    "% Passing Reading": [pct_reading_pass],
    "% Overall Passing": [pct_overall_pass]
})

## 3. Set District Summary Report Format(Style)
district_style = {
    "Total Students": '{:,}', 
    "Total Budget": '${:,.2f}',
    "Average Math Score": '{:.2f}',
    "Average Reading Score": '{:.2f}',
    "% Passing Math": '{:.2%}',
    "% Passing Reading": '{:.2%}',
    "% Overall Passing": '{:.2%}'
}

## 4. Display District Summary
district_summary.style.format(district_style)

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,65.17%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [5]:
## SCHOOL SUMMARY

## 1. Build School Summary DataFrame from School Data

school_summary = school_data[["School Name", "School Type", "Total Students", "Total School Budget"]]
## Set School Name as index
school_summary = school_summary.set_index("School Name", drop=True)
## Remove index name
school_summary.index.name = None

## 2. Compute and append columns

## Per Student Budget
school_summary["Per Student Budget"] = school_summary["Total School Budget"] / school_summary["Total Students"]
## Average Math Score
school_summary["Average Math Score"] = school_analysis.groupby("School Name")["Math Score"].mean()
## Average Reading Score
school_summary["Average Reading Score"] = school_analysis.groupby("School Name")["Reading Score"].mean()
## Passing Math Percentage
school_summary["% Passing Math"] = school_analysis.groupby("School Name")["Math Pass"].sum() / school_summary["Total Students"]
## Passing Reading Percentage
school_summary["% Passing Reading"] = school_analysis.groupby("School Name")["Reading Pass"].sum() / school_summary["Total Students"]
## Overall Passing Percentage (students that passed math and reading)
school_summary["% Overall Passing"] = school_analysis.groupby("School Name")["Overall Pass"].sum() / school_summary["Total Students"]

## 3. Set School Summary Report Format(Style)
school_style = {
    "Total Students": '{:,}',
    "Total School Budget": '${:,.2f}',
    "Per Student Budget": '${:,.2f}',
    "Average Math Score": '{:.2f}',
    "Average Reading Score": '{:.2f}',
    "% Passing Math": '{:.2%}',
    "% Passing Reading": '{:.2%}',
    "% Overall Passing": '{:.2%}'
}

## 4. Display School Summary (alphabetical order)
school_summary.sort_index(axis=0).style.format(school_style)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,66.68%,81.93%,54.64%
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,54.29%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51%,96.25%,89.23%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,90.54%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [6]:
## TOP PERFORMING SCHOOLS

## Sort by "% Overall Passing" & Display top 5
school_summary.sort_values("% Overall Passing", ascending=False).head(5).style.format(school_style)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42,83.85,93.27%,97.31%,90.95%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,83.99,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,90.54%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [7]:
## BOTTOM PERFORMING SCHOOLS

## Sort by "% Overall Passing" & Display bottom 5
school_summary.sort_values("% Overall Passing", ascending=True).head(5).style.format(school_style)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84,80.74,66.37%,80.22%,52.99%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%


## Math Scores by Grade

* Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [8]:
## SCORES BY GRADE

## Get Math & Reading Scores by Grade
grade_9 = school_analysis[["School Name", "Math Score", "Reading Score"]].loc[school_analysis["Grade"] == "9th"]
grade_10 = school_analysis[["School Name", "Math Score", "Reading Score"]].loc[school_analysis["Grade"] == "10th"]
grade_11 = school_analysis[["School Name", "Math Score", "Reading Score"]].loc[school_analysis["Grade"] == "11th"]
grade_12 = school_analysis[["School Name", "Math Score", "Reading Score"]].loc[school_analysis["Grade"] == "12th"]

In [9]:
## MATH SCORES BY GRADE

## Build Math Scores by Grade DataFrame
math_by_grade = pd.DataFrame({
    "9th": grade_9.groupby("School Name")["Math Score"].mean(),
    "10th": grade_10.groupby("School Name")["Math Score"].mean(),
    "11th": grade_11.groupby("School Name")["Math Score"].mean(),
    "12th": grade_12.groupby("School Name")["Math Score"].mean()
})

## Remove index name
math_by_grade.index.name = None

## Display Math Scores by Grade (alphabetical order & format)
math_by_grade.sort_index(axis=0).style.format('{:.2f}')

,9th,10th,11th,12th
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86
Pena High School,83.63,83.37,84.33,84.12


## Reading Scores by Grade 

* Perform the same operations as above for reading scores

In [10]:
## READING SCORES BY GRADE

## Build Reading Scores by Grade DataFrame
reading_by_grade = pd.DataFrame({
    "9th": grade_9.groupby("School Name")["Reading Score"].mean(),
    "10th": grade_10.groupby("School Name")["Reading Score"].mean(),
    "11th": grade_11.groupby("School Name")["Reading Score"].mean(),
    "12th": grade_12.groupby("School Name")["Reading Score"].mean()
})

## Remove index name
reading_by_grade.index.name = None

## Display Reading Scores by Grade (alphabetical order & format)
reading_by_grade.sort_index(axis=0).style.format('{:.2f}')

,9th,10th,11th,12th
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23
Pena High School,83.81,83.61,84.34,84.59


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [11]:
## Set Binning Report Format(Style)
binning_style = {
    "Average Math Score": '{:.2f}',
    "Average Reading Score": '{:.2f}',
    "% Passing Math": '{:.2%}',
    "% Passing Reading": '{:.2%}',
    "% Overall Passing": '{:.2%}'
}

In [12]:
## SCORES BY SCHOOL SPENDING

## Compute Min & Max Spending (Per Student)
## print(school_summary["Per Student Budget"].min(), school_summary["Per Student Budget"].max())

## Set Spending Ranges (Per Student) & Labels
spending_bins = [0, 585, 630, 645, 680]
spending_labels = ["<$585", "$585-630", "$630-645", "$645-680" ]

## Add Spending Ranges (Per Student) column
school_summary["Spending Ranges (Per Student)"] = pd.cut(school_summary["Per Student Budget"], bins=spending_bins, labels=spending_labels, right=False, include_lowest=False)

## Group by Spending Range (Per Student)
spending_group = school_summary.groupby("Spending Ranges (Per Student)")[[
    "Average Math Score",
    "Average Reading Score",
    "% Passing Math",
    "% Passing Reading",
    "% Overall Passing"
]].mean()

## Display Spending Range (Per Student) Group
spending_group.style.format(binning_style)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46%,96.61%,90.37%
$585-630,81.90,83.16,87.13%,92.72%,81.42%
$630-645,78.52,81.62,73.48%,84.39%,62.86%
$645-680,77.00,81.03,66.16%,81.13%,53.53%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [13]:
## SCORES BY SCHOOL SIZE

## Compute Min & Max Total Students
## print(school_summary["Total Students"].min(), school_summary["Total Students"].max())

## Set School Size Ranges & Labels
size_bins = [0, 1000, 2000, 5000]
size_labels = ["Small (<$1000)", "Medium (1000-2000)", "Large (2000-5000)"]

## Add School Size column
school_summary["School Size"] = pd.cut(school_summary["Total Students"], bins=size_bins, labels=size_labels, right=False, include_lowest=False)

## Group by School Size
size_group = school_summary.groupby("School Size")[[
    "Average Math Score",
    "Average Reading Score",
    "% Passing Math",
    "% Passing Reading",
    "% Overall Passing"
]].mean()

## Display School Size Group
size_group.style.format(binning_style)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<$1000),83.82,83.93,93.55%,96.10%,89.88%
Medium (1000-2000),83.37,83.86,93.60%,96.79%,90.62%
Large (2000-5000),77.75,81.34,69.96%,82.77%,58.29%


## Scores by School Type

* Perform the same operations as above, based on school type

In [14]:
## SCORES BY SCHOOL TYPE

## Group by School Type
type_group = school_summary.groupby("School Type")[[
    "Average Math Score",
    "Average Reading Score",
    "% Passing Math",
    "% Passing Reading",
    "% Overall Passing"
]].mean()

## Display School Type Group
type_group.style.format(binning_style)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,93.62%,96.59%,90.43%
District,76.96,80.97,66.55%,80.80%,53.67%


## Key Insights

Three trends can be observed from the *School Spending*, *School Size* and *School Type* groups:

1. **Students from schools that have a lower budget per student (less than $585) perform better**, as the budget per student increases, the student's performance decreases. This seems very counterintuitive, and it might be valuable to search for the underlying causes: spending efficiency, school board autonomy to allocate funds, etc. The drop in performance of the higher spending range (+35% overall passing) relative to the lower range is very significant.<br><br>

2. **Students from Small and Medium size schools (less than 2,000 students) perform better** than large schools (2,000 up to 5,000 students). The performance difference between Small and Medium schools seems negligible, but the difference between these and Large schools (+30% overall passing) is quite notable.<br><br>

3. **Students from Charter schools perform better** than students from District schools. The difference in performance (+35% overall passing) is meaningful, and in the same order of magnitude between lower vs higher spending per student. This would suggest to further explore the relationship between the three variables (spending, size and type) in order to explain these
trends. 